# ETL Processes
We Use this notebook to develop our  ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.
1. Single row insertion commands were used first for insertion and also given for reference. But we have used multi row insertion (BULK insertion) using COPY command of  postgreSQL.
2. Please refer create_tables.py for all create and insert commands.
3. To check the null values insertion and their data checkup, we have used all files in the directory "data/song_data/A/A/A"  for songs and artists table.
4. We have used temporary tables for songs, artists, time, users for the BULK insertion of data so to include ON CONFLICT clause.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
def process_data(filepath):
    """iterate over all files in a path
    
    Parameters:
    ----------- 
    filepath: path of the .json files of which 
              data to be inserted into table
    """
    
    # get all files matching extension from directory 
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))  
    return all_files

In [5]:
file_names = process_data("data/song_data/A/A/A")

df = pd.DataFrame()
for each in file_names:
    df = df.append(pd.read_json(each, lines=True), ignore_index=True)

11 files found in data/song_data/A/A/A


In [6]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
2,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
4,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# --- for a single row extraction
#song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values[0]
#song_data = list(song_data)

In [8]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]]
song_data

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39628
2,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
5,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),ARKFYS91187B98E58F,0,267.70240
6,SOMJBYD12A6D4F8557,Keepin It Real (Skit),ARD0S291187B9B7BF5,0,114.78159
7,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506
8,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
9,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
#--- for single row insertion
#cur.execute(song_table_insert, song_data)
#conn.commit()

In [13]:
#----- Multi row insertion------#
# 1. Write DataFrame into csv file
song_data.to_csv('/home/workspace/data/song_data/song_data.csv', index=False, header=False)

# 2. Create Temporary table
cur.execute(temp_songs_table_create)
# Test data
cur.execute("select count(*) from songs_temp")
row = cur.fetchone()
print(f'row counts in songs_temp table before insertion is {row}')

# 3. Load data into temp file
temp_file_path = '/home/workspace/data/song_data/song_data.csv'

# 4. Insert data into temporary table
cur.execute(song_temp_table_insert_)
# check data in temp table
cur.execute("select count(*) from songs_temp")
row = cur.fetchone()
print(f'row counts in songs_temp table is {row}')

# 5. insert data into songs table
cur.execute(song_table_insert)

# check data in temp table
cur.execute("select count(*) from songs")
row = cur.fetchone()
print(f'row counts in songs table is {row}')

# 6. Delete temp file and temp table
if os.path.exists(temp_file_path):
    os.remove(temp_file_path)
else:
    print("The file does not exist")
    
cur.execute("Drop Table songs_temp;")
conn.commit()

row counts in songs_temp table before insertion is (22,)
row counts in songs_temp table is (33,)
row counts in songs table is (11,)


#### Testing

In [14]:
query_song = "SELECT * FROM songs"
try:
    cur.execute(query_song)
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, Decimal('267.7024'))
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, Decimal('114.78159'))
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, Decimal('218.77506'))
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, Decimal('233.40363'))
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, Decimal('209.60608'))
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA

Pleasd also refer `test.ipynb` to see if successfully records are added to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
# ---or a single row insertion
# artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]
# artist_data = list(artist_data)

In [16]:
artists_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artists_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
2,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
5,ARKFYS91187B98E58F,Jeff And Sheri Easter,,NaN,NaN
6,ARD0S291187B9B7BF5,Rated R,Ohio,NaN,NaN
7,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955
8,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
9,ARXR32B1187FB57099,Gob,,NaN,NaN


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
#----- Multi row insertion------#
# 1. Write DataFrame into csv file
artists_data.to_csv('/home/workspace/data/song_data/artists_data.csv', index=False, header=False)

# 2. Create Temporary table
cur.execute(temp_artists_table_create)
# Test data
cur.execute("select count(*) from artists_temp")
row = cur.fetchone()
print(f'row counts in artists_temp table before insertion is {row}')

# 3. Load data into temp file
temp_file_path = '/home/workspace/data/song_data/artists_data.csv'


# 4. Insert data into temporary table
cur.execute(artists_temp_table_insert_)

# check data in temp table
cur.execute("select count(*) from artists_temp")
row = cur.fetchone()
print(f'row counts in artists_temp table is {row}')

# 5. insert data into artists table
cur.execute(artist_table_insert)

# check data in artists table
cur.execute("select count(*) from artists")
row = cur.fetchone()
print(f'row counts in artists table is {row}')

# 6. Delete temp file and temp table
if os.path.exists(temp_file_path):
    os.remove(temp_file_path)
else:
    print("The file does not exist")
     
cur.execute("Drop Table artists_temp;")
conn.commit()

row counts in artists_temp table before insertion is (0,)
row counts in artists_temp table is (11,)
row counts in artists table is (11,)


## Testing and data quality check
### Data quality check:
1. Nulls are handeled properly. Below is the image of with PGadmin
![Image](./images/Data_check_1.png)

In [20]:
query_artists = "SELECT * FROM artists"
try:
    cur.execute(query_artists)
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)
('ARNTLGG11E2835DDB9', 'Clp', None, None, None)
('AR8ZCNI1187B9A069B', 'Planet P Project', None, None, None)
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', None, None)
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)
('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', None, None, None)
('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', None, None)
('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955)
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None)
('ARXR32B1187FB57099', 'Gob', None, None, None)
('ARKRRTF1187B9984DA', 'Sonora Santanera', None, None, None)


Please also Refer `test.ipynb` to see if successfully added records to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [21]:
# get a list of all log JSON files in data/log_data
log_files = get_files("data/log_data/")

In [22]:
filepath = log_files[0]

In [23]:
df = pd.read_json(filepath, lines=True)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [24]:
# Filter records by NextSong action
df_ = df[df['page'] == 'NextSong']

In [25]:
# Convert the ts timestamp column to datetime
t = pd.to_datetime(df_["ts"], unit = "ms")
timestamp_ = list(df_["ts"])
hour_ = list(t.dt.hour)
day_ = list(t.dt.day) 
week_ = list(t.dt.week) 
month_ = list(t.dt.month) 
year_ = list(t.dt.year)
weekday_ = list(t.dt.weekday) 
time_data = [timestamp_, hour_, day_, week_, month_, year_, weekday_]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [26]:
time_dict = {label :time_data[i]  for i, label in enumerate(column_labels)}
time_df = pd.DataFrame(time_dict)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543537327796,0,30,48,11,2018,4
1,1543540121796,1,30,48,11,2018,4
2,1543540368796,1,30,48,11,2018,4
3,1543540625796,1,30,48,11,2018,4
4,1543540856796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [27]:
#---  single row insertion
#for i, row in time_df.iterrows():
#    cur.execute(time_table_insert, list(row))
#    conn.commit()

In [28]:
#----- Multi row insertion------#

# 1. Write DataFrame into csv file
time_df.to_csv('/home/workspace/data/log_data/time_data.csv', index=False, header=False)

# 2. Create Temporary table
cur.execute(temp_time_table_create)
# Test data
cur.execute("select count(*) from time_temp")
row = cur.fetchone()
print(f'row counts in time_temp table before insertion is {row}')

row counts in time_temp table before insertion is (0,)


In [29]:
# 2. Load data into time table
temp_file_path = '/home/workspace/data/time_data/time_data.csv'

In [31]:
# 3. insert data into time temp table
cur.execute(time_temp_table_insert)

In [32]:
# 4. insert data into time table
cur.execute(time_table_insert)

In [45]:
conn.commit()

#### Testing & data check

In [33]:
query_time = "SELECT count(*) FROM time"

# check data in time table
cur.execute(query_time)
row = cur.fetchone()
print(f'row counts in time table is {row}')

try:
    cur.execute(query_time)
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()

# to check same number of distinct rows in Dataframe and Table
print(row[0] == time_df["start_time"].nunique())

row counts in time table is (330,)
True


In [34]:
# to check first five data
try:
    cur.execute('SELECT * from time')
except psycopg2.Error as e:
    print(e)
for i in range(5):
    row = cur.fetchone()
    print(row)

(datetime.datetime(2018, 11, 30, 0, 22, 7), 0, 30, 48, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 8, 41), 1, 30, 48, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 12, 48), 1, 30, 48, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 17, 5), 1, 30, 48, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 20, 56), 1, 30, 48, 11, 2018, 4)


Please also Refer `test.ipynb` to see if successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [35]:
user_df_cols = ["userId", "firstName", "lastName", "gender", "level"]
user_df = df_.loc[:, user_df_cols]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [36]:
# --single row insertion
#for i, row in user_df.iterrows():
#    cur.execute(user_table_insert, list(row))
#    conn.commit()

In [37]:
# ---- multi row insertion
# 1. Write DataFrame into csv file
user_df.to_csv('/home/workspace/data/log_data/users_data.csv', index=False, header=False)

# 2. Create temporary table
cur.execute(temp_users_table_create)

In [38]:
# 3. users_temp table check
cur.execute(users_temp_table_insert)
cur.execute("select count(*) from users_temp")
row = cur.fetchone()
print(row)

(330,)


In [40]:
# 4. insert into users table
cur.execute(user_table_insert)

In [44]:
conn.commit()

In [41]:
# 5. users table check
cur.execute("select count(*) from users")
row = cur.fetchone()
print(row)

(22,)


#### Data Check and Testing

In [42]:
query_users = "SELECT count(*) FROM users"
try:
    cur.execute(query_users)
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()

# to check same number of distinct rows
print(row[0] == user_df["userId"].nunique())

True


In [43]:
# to check first five data
try:
    cur.execute('SELECT * from users')
except psycopg2.Error as e:
    print(e)
for i in range(5):
    row = cur.fetchone()
    print(row)

(91, 'Jayden', 'Bell', 'M', 'free')
(73, 'Jacob', 'Klein', 'M', 'paid')
(86, 'Aiden', 'Hess', 'M', 'free')
(24, 'Layla', 'Griffin', 'F', 'paid')
(26, 'Ryan', 'Smith', 'M', 'free')


Please also Refer `test.ipynb` to see if successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [61]:
for index, row in df_.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

### Test and data check

In [ ]:
# to check first five data
try:
    cur.execute('SELECT * from songplays')
except psycopg2.Error as e:
    print(e)
for i in range(5):
    row = cur.fetchone()
    print(row)

Please also refer `test.ipynb` to see if successfully added records to this table. `test.ipynb` to see if successfully added records to the table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`

We see that we have successfully inserted data into all tables of our data model. Now we implement it into etl.py